In [43]:
pip install keras

In [44]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [45]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout

In [46]:
import tensorflow as tf

dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))

print(dataset1.element_spec)

TensorSpec(shape=(10,), dtype=tf.float32, name=None)


In [47]:
df = pd.read_csv("IsraeliLottery.csv")
df.head()

,Game,Date,A,B,C,D,E,F
0,6801,03/09/1968,3,14,18,22,25,33
1,6802,10/09/1968,13,20,23,29,32,34
2,6803,17/09/1968,8,12,26,27,34,38
3,6804,24/09/1968,1,14,17,26,35,39
4,6805,01/10/1968,1,7,8,9,11,30


In [48]:
df.tail()

,Game,Date,A,B,C,D,E,F
4042,3467,14/05/2022,9,16,17,21,26,29
4043,3468,17/05/2022,1,4,5,24,31,32
4044,3469,19/05/2022,1,8,18,25,29,30
4045,3470,21/05/2022,3,4,5,15,24,33
4046,3471,24/05/2022,6,10,13,20,23,35


In [49]:
print(df.shape)

(4047, 8)


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4047 entries, 0 to 4046
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Game    4047 non-null   int64 
 1   Date    4047 non-null   object
 2   A       4047 non-null   int64 
 3   B       4047 non-null   int64 
 4   C       4047 non-null   int64 
 5   D       4047 non-null   int64 
 6   E       4047 non-null   int64 
 7   F       4047 non-null   int64 
dtypes: int64(7), object(1)
memory usage: 253.1+ KB


In [51]:
df.describe()

,Game,A,B,C,D,E,F
count,4047.000000,4047.000000,4047.000000,4047.000000,4047.000000,4047.000000,4047.000000
mean,4695.399061,5.698542,11.428713,17.128737,22.948357,28.795404,34.635285
std,3106.179305,4.635402,5.988067,6.698026,6.820599,6.616360,5.910514
min,1035.000000,1.000000,2.000000,3.000000,4.000000,7.000000,13.000000
25%,2046.500000,2.000000,7.000000,12.000000,18.000000,24.000000,31.000000
50%,3058.000000,4.000000,10.000000,17.000000,23.000000,29.000000,35.000000
75%,8011.500000,8.000000,15.000000,22.000000,28.000000,33.000000,38.000000
max,9934.000000,31.000000,37.000000,44.000000,46.000000,48.000000,49.000000


In [52]:
df.drop(['Game', 'Date'], axis=1, inplace=True)
df.head()

,A,B,C,D,E,F
0,3,14,18,22,25,33
1,13,20,23,29,32,34
2,8,12,26,27,34,38
3,1,14,17,26,35,39
4,1,7,8,9,11,30


In [53]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [54]:
transformed_df.head()


,0,1,2,3,4,5
0,-0.582231,0.429455,0.130094,-0.139060,-0.573710,-0.276708
1,1.575345,1.431572,0.876674,0.887369,0.484404,-0.107497
2,0.496557,0.095416,1.324623,0.594104,0.786722,0.569346
3,-1.013746,0.429455,-0.019223,0.447471,0.937882,0.738557
4,-1.013746,-0.739681,-1.363068,-2.045287,-2.689939,-0.784341


In [55]:
# All our games
number_of_rows = df.values.shape[0]
number_of_rows

4047

In [56]:
# Balls counts
number_of_features = df.values.shape[1]
number_of_features

6

In [57]:
# Amount of games we need to take into consideration for prediction
window_length = 7
window_length


7

In [58]:
X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [59]:
X.shape

(4040, 7, 6)

In [60]:
y.shape


(4040, 6)

In [61]:
X[0]


array([[-0.58223113,  0.429455  ,  0.1300936 , -0.13906023, -0.57371015,
        -0.27670816],
       [ 1.57534546,  1.43157169,  0.87667445,  0.88736948,  0.4844041 ,
        -0.10749723],
       [ 0.49655716,  0.0954161 ,  1.32462296,  0.59410385,  0.78672246,
         0.56934648],
       [-1.01374645,  0.429455  , -0.01922257,  0.44747103,  0.93788164,
         0.7385574 ],
       [-1.01374645, -0.73968114, -1.36306809, -2.04528682, -2.68993865,
        -0.78434094],
       [ 0.2807995 , -0.40564224, -1.06443575, -0.8722243 , -0.87602851,
        -0.10749723],
       [-1.01374645, -0.90670059,  0.87667445,  0.88736948,  0.63556328,
        -0.10749723]])

In [62]:
y[0]


array([-0.79798879,  0.26243555,  0.42872594,  0.1542054 , -0.57371015,
       -1.29197372])

In [63]:
y[1]


array([-0.58223113, -0.40564224,  1.32462296,  1.32726792,  0.78672246,
        0.23092462])

In [64]:
# Init RNN
model = Sequential()
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [65]:
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [66]:
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = True)))

In [67]:
model.add(Bidirectional(LSTM(240, input_shape = (window_length, number_of_features), return_sequences = False)))

In [68]:
model.add(Dense(59))


In [69]:
model.add(Dense(number_of_features))


In [72]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

In [73]:
model.fit(x=X, y=y, batch_size=150, epochs=300, verbose=2)

Epoch 1/300
27/27 - 58s - loss: 0.8821 - accuracy: 0.2780
Epoch 2/300
27/27 - 13s - loss: 0.8784 - accuracy: 0.2760
Epoch 3/300
27/27 - 14s - loss: 0.8773 - accuracy: 0.2819
Epoch 4/300
27/27 - 16s - loss: 0.8772 - accuracy: 0.2792
Epoch 5/300
27/27 - 15s - loss: 0.8793 - accuracy: 0.2775
Epoch 6/300
27/27 - 19s - loss: 0.8772 - accuracy: 0.2782
Epoch 7/300
27/27 - 23s - loss: 0.8775 - accuracy: 0.2809
Epoch 8/300
27/27 - 19s - loss: 0.8763 - accuracy: 0.2772
Epoch 9/300


KeyboardInterrupt: 

In [ ]:
to_predict = df.tail(8)
to_predict

In [ ]:
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

In [ ]:
prediction = df.tail(1)
prediction

In [ ]:
to_predict = np.array(to_predict)
to_predict

In [ ]:
scaled_to_predict = scaler.transform(to_predict)
scaled_to_predict

In [ ]:
y_pred = model.predict(np.array([scaled_to_predict]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

In [ ]:
prediction = np.array(prediction)
print("The actual numbers in the last lottery game were:", prediction[0])